# Création du dataset

Ce notebook sert à créer un dataset d'activations de neuronnes. Pour créer ce dataset la première étape est d'entraîner un MLP sur le dataset MNIST, une fois le modèle entraîné on va faire de la prédiction tout en récupérant l'activation des neuronnes. Et enfin on sauvegarde ces activations dans un fichier CSV contenant les variables suivantes :
- `nb_true` : Vrai nombre
- `nb_pred` : Nombre qui a été prédit par le modèle
- `activation1` : activation du neuronne 1 
- `activation2` : activation du neuronne 2
- ...

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import csv

## Configuration

In [2]:
# Cette variable indique les nombres de neuronnes et le nombre de couche caché 
# ici [128, 64] indique 2 couches caché avec une couche de 128 neuronnes et une autres de 64 
hidden_sizes = [128, 64] 

# Nombre d'épochs de l'entraînement
epochs = 5

# learning_rate pour l'apprentissage
learning_rate = 0.001

# nom du fichier de sortie, IMPORTANT : doit être un fichier csv ex : "activations.csv"
output_file = "../../data/activations/activations.csv"

# taille des batchs
batch_size = 64

## Préparation des données MNIST

In [3]:
# Préparer les données MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normaliser entre -1 et 1
])

train_dataset = datasets.MNIST(root='../../data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='../../data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Création du modèle MLP

In [4]:
class MLP(nn.Module):
    def __init__(self, input_size=784, hidden_sizes=[128, 64], output_size=10):
        super(MLP, self).__init__()
        self.hidden_layers = nn.ModuleList()
        in_features = input_size
        for h in hidden_sizes:
            self.hidden_layers.append(nn.Linear(in_features, h))
            in_features = h
        self.output_layer = nn.Linear(in_features, output_size)
        self.activations = []

    def forward(self, x):
        self.activations = []  # Réinitialiser les activations à chaque passage
        x = x.view(x.size(0), -1)  # Aplatir l'image en vecteur
        for layer in self.hidden_layers:
            x = layer(x)
            x = torch.relu(x)
            self.activations.append(x.clone())  # Sauvegarder les activations après ReLU
        x = self.output_layer(x)
        return x

In [5]:
model = MLP(hidden_sizes=hidden_sizes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Entraînement du modèle

In [6]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Époch {epoch+1}/{epochs}, Perte: {running_loss/len(train_loader)}")

Époch 1/5, Perte: 0.39569791773361945
Époch 2/5, Perte: 0.1842050273964273
Époch 3/5, Perte: 0.13582855409809522
Époch 4/5, Perte: 0.10901505915797564
Époch 5/5, Perte: 0.09439217581995514


## Evaluation et prédiction

On évalue le modèle sur le jeux de données de test et on récupère les activations de neuronnes.



In [7]:
model.eval()
all_activations = []
all_predictions = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)
        all_predictions.extend(predictions.numpy())
        all_labels.extend(labels.numpy())
        
        # Collecter les activations des couches cachées
        batch_activations = [activation.numpy() for activation in model.activations]
        # Transposer pour obtenir (échantillons, activations par couche)
        batch_activations = np.concatenate(batch_activations, axis=1)  # Concatène les activations des couches
        all_activations.extend(batch_activations)

## sauvegarde des activations

In [8]:
# Préparer les données pour le CSV
csv_data = []
for true_label, pred_label, activation in zip(all_labels, all_predictions, all_activations):
    # Ajouter les données sous forme [nb_true, nb_pred, activation1, activation2, ...]
    csv_data.append([true_label, pred_label] + activation.tolist())

# Sauvegarder les données dans un fichier CSV
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Écrire l'en-tête du fichier
    header = ["nb_true", "nb_pred"] + [f"activation{i+1}" for i in range(len(csv_data[0]) - 2)]
    writer.writerow(header)
    # Écrire les données
    writer.writerows(csv_data)

print(f"Les activations ont été sauvegardées dans {output_file}.")

Les activations ont été sauvegardées dans ../../data/activations/activations.csv.
